# 0. 패키지 불러오기

In [6]:
import pandas as pd
import numpy as np
from IPython.display import display as dp
import warnings
import matplotlib
import matplotlib.pyplot as plt
from time import time

# 스케일링 => KNN은 거리 기반이므로 이상치에 민감하니 StandardScaler 사용
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix,f1_score, accuracy_score

warnings.filterwarnings('ignore')
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 검증과 테스트 데이터셋은 피처 스케일링되어 사용되므로 먼저 정의하였다.
validation =pd.read_csv('../data/validation_encoding.csv', index_col=0)
test =pd.read_csv('../data/test_encoding.csv', index_col=0)

X_val, y_val = validation.iloc[:,:-1], validation.iloc[:,-1] # 그대로 사용됨. 
X_test, y_test = test.iloc[:,:-1], test.iloc[:,-1] # 그대로 사용됨..

dp(validation,test)
dp(X_val, y_val.to_frame() , X_test, y_test.to_frame())

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3,TARGET
73378,3,4.0,10,750,22.0,3.711225,0.000000,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,2
71524,8,0.0,10,329,12.0,2.734892,0.000000,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1
38781,3,0.0,4,649,10.0,1.294901,0.000000,0.0,0.0,335.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
35212,10,5.0,6,628,20.0,1.337039,0.000000,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
56852,8,2.0,6,1846,36.0,2.158072,415.666667,0.0,0.0,205.0,...,1.0,0.0,0.0,0.0,0.0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,9,6.0,6,2148,38.0,2.184994,3.125000,0.0,0.0,220.0,...,1.0,0.0,0.0,0.0,0.0,1,0,0,0,0
36543,4,6.0,1,1827,51.0,2.204480,0.000000,0.0,0.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0
70132,11,4.0,2,1349,36.0,1.873360,1.142857,0.0,0.0,285.0,...,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
58097,9,0.0,12,329,12.0,3.002181,19.625000,0.0,0.0,165.0,...,1.0,0.0,0.0,0.0,0.0,1,1,0,1,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3,TARGET
12635,9,3.0,10,1050,32.0,1.254874,0.000000,0.0,0.000000,105.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
22195,5,5.0,10,1447,41.0,1.494139,5.142857,0.0,0.000000,220.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
42119,11,6.0,4,1348,6.0,2.063934,0.000000,0.0,0.000000,30.0,...,0.0,0.0,0.0,1.0,0.0,1,0,0,0,2
34954,1,2.0,1,746,21.0,1.746713,22.000000,35.0,48.333333,210.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
65122,4,3.0,2,426,13.0,2.079781,0.555556,0.0,0.000000,220.0,...,1.0,0.0,0.0,0.0,0.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41734,11,5.0,9,936,28.0,0.650350,0.000000,0.0,0.000000,320.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1
16135,6,1.0,8,1938,52.0,2.386494,24.250000,0.0,0.000000,200.0,...,1.0,0.0,0.0,0.0,0.0,0,0,1,0,1
65530,1,2.0,9,1148,29.0,0.446058,28.875000,34.0,144.000000,345.0,...,0.0,0.0,0.0,1.0,0.0,0,0,1,1,1
80594,1,5.0,6,36,46.0,2.517878,23.000000,15.8,22.666667,310.0,...,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3
73378,3,4.0,10,750,22.0,3.711225,0.000000,0.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
71524,8,0.0,10,329,12.0,2.734892,0.000000,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
38781,3,0.0,4,649,10.0,1.294901,0.000000,0.0,0.0,335.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
35212,10,5.0,6,628,20.0,1.337039,0.000000,0.0,0.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
56852,8,2.0,6,1846,36.0,2.158072,415.666667,0.0,0.0,205.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,9,6.0,6,2148,38.0,2.184994,3.125000,0.0,0.0,220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0
36543,4,6.0,1,1827,51.0,2.204480,0.000000,0.0,0.0,215.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
70132,11,4.0,2,1349,36.0,1.873360,1.142857,0.0,0.0,285.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
58097,9,0.0,12,329,12.0,3.002181,19.625000,0.0,0.0,165.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1,1,0,1


,TARGET
73378,2
71524,1
38781,0
35212,0
56852,1
...,...
23050,0
36543,0
70132,0
58097,0


,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,...,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지_0,범죄발생지_1,범죄발생지_2,범죄발생지_3
12635,9,3.0,10,1050,32.0,1.254874,0.000000,0.0,0.000000,105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
22195,5,5.0,10,1447,41.0,1.494139,5.142857,0.0,0.000000,220.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
42119,11,6.0,4,1348,6.0,2.063934,0.000000,0.0,0.000000,30.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,0
34954,1,2.0,1,746,21.0,1.746713,22.000000,35.0,48.333333,210.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
65122,4,3.0,2,426,13.0,2.079781,0.555556,0.0,0.000000,220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41734,11,5.0,9,936,28.0,0.650350,0.000000,0.0,0.000000,320.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0
16135,6,1.0,8,1938,52.0,2.386494,24.250000,0.0,0.000000,200.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,1,0
65530,1,2.0,9,1148,29.0,0.446058,28.875000,34.0,144.000000,345.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1,1
80594,1,5.0,6,36,46.0,2.517878,23.000000,15.8,22.666667,310.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0


,TARGET
12635,1
22195,1
42119,2
34954,0
65122,0
...,...
41734,1
16135,1
65530,1
80594,0


# 1 베이스 모델 강화하기
파라미터 공간을 크게 늘려서 돌려보기

In [7]:
# 피처 스케일링에 사용될 scaler
scaler = MinMaxScaler()

# base model의 학습 데이터
base_train = pd.read_csv('../data/base_train_encoding.csv', index_col=0)
X_train_base, y_train_base = base_train.iloc[:,:-1], base_train.iloc[:,-1]

X_train_base_scaled = scaler.fit_transform(X_train_base)

# 어떤 모델이건 사용될 검증/테스트 데이터셋 # 이게 아니라는 것. 훈련데이터셋이 바뀔 때마다 새롭게 transform 해야함. 이건 그냥 base_model의 경우임.
X_val_base_scaled = scaler.transform(X_val)
X_test_base_scaled = scaler.transform(X_test)

X_train_base_scaled.shape # pd.DataFrame(X_train_base_scaled, index = X_train_base.index, columns = X_train_base.columns)

(50643, 21)

In [3]:
from time import time
start = time()

knn = KNeighborsClassifier()

# 적절한 k값을 위한 과정
from sklearn.model_selection import GridSearchCV
# from skopt import BayesSearchCV
# params = {
#     'n_neighbors' : (1,30),
#     'weights' : ['uniform'],
#     'p' : [2]
# } # 기존 탐색
params = {
    'n_neighbors' : range(1, 50),
    'weights' : ['uniform', 'distance'],
    'p' : [1, 2]
}
grid_search = GridSearchCV(knn, params, cv = 5, verbose = 2)
grid_search.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
k_knn_base = grid_search.best_params_
print(k_knn_base)

Fitting 5 folds for each of 196 candidates, totalling 980 fits
[CV] END ................n_neighbors=1, p=1, weights=uniform; total time=  14.7s
[CV] END ................n_neighbors=1, p=1, weights=uniform; total time=  14.7s
[CV] END ................n_neighbors=1, p=1, weights=uniform; total time=  14.9s
[CV] END ................n_neighbors=1, p=1, weights=uniform; total time=  15.7s
[CV] END ................n_neighbors=1, p=1, weights=uniform; total time=  16.1s
[CV] END ...............n_neighbors=1, p=1, weights=distance; total time=  15.2s
[CV] END ...............n_neighbors=1, p=1, weights=distance; total time=  15.4s
[CV] END ...............n_neighbors=1, p=1, weights=distance; total time=  15.2s
[CV] END ...............n_neighbors=1, p=1, weights=distance; total time=  15.1s
[CV] END ...............n_neighbors=1, p=1, weights=distance; total time=  15.0s
[CV] END ................n_neighbors=1, p=2, weights=uniform; total time=   7.5s
[CV] END ................n_neighbors=1, p=2, w

[CV] END ................n_neighbors=6, p=1, weights=uniform; total time=  16.3s
[CV] END ................n_neighbors=6, p=1, weights=uniform; total time=  15.2s
[CV] END ................n_neighbors=6, p=1, weights=uniform; total time=  14.6s
[CV] END ................n_neighbors=6, p=1, weights=uniform; total time=  14.6s
[CV] END ...............n_neighbors=6, p=1, weights=distance; total time=  14.5s
[CV] END ...............n_neighbors=6, p=1, weights=distance; total time=  13.7s
[CV] END ...............n_neighbors=6, p=1, weights=distance; total time=  13.1s
[CV] END ...............n_neighbors=6, p=1, weights=distance; total time=  13.3s
[CV] END ...............n_neighbors=6, p=1, weights=distance; total time=  13.5s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   9.0s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   9.4s
[CV] END ................n_neighbors=6, p=2, weights=uniform; total time=   9.2s
[CV] END ................n_n

[CV] END ...............n_neighbors=11, p=1, weights=uniform; total time=  14.0s
[CV] END ...............n_neighbors=11, p=1, weights=uniform; total time=  14.0s
[CV] END ..............n_neighbors=11, p=1, weights=distance; total time=  13.0s
[CV] END ..............n_neighbors=11, p=1, weights=distance; total time=  13.3s
[CV] END ..............n_neighbors=11, p=1, weights=distance; total time=  13.3s
[CV] END ..............n_neighbors=11, p=1, weights=distance; total time=  13.5s
[CV] END ..............n_neighbors=11, p=1, weights=distance; total time=  13.3s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   8.8s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   8.8s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   8.8s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   8.7s
[CV] END ...............n_neighbors=11, p=2, weights=uniform; total time=   9.1s
[CV] END ..............n_nei

[CV] END ..............n_neighbors=16, p=1, weights=distance; total time=  16.4s
[CV] END ..............n_neighbors=16, p=1, weights=distance; total time=  17.0s
[CV] END ..............n_neighbors=16, p=1, weights=distance; total time=  16.7s
[CV] END ..............n_neighbors=16, p=1, weights=distance; total time=  17.0s
[CV] END ..............n_neighbors=16, p=1, weights=distance; total time=  17.1s
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time=  10.0s
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time=  10.5s
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time=  10.6s
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time=  10.3s
[CV] END ...............n_neighbors=16, p=2, weights=uniform; total time=  10.7s
[CV] END ..............n_neighbors=16, p=2, weights=distance; total time=   9.8s
[CV] END ..............n_neighbors=16, p=2, weights=distance; total time=  10.1s
[CV] END ..............n_nei

[CV] END ..............n_neighbors=21, p=1, weights=distance; total time=  15.4s
[CV] END ..............n_neighbors=21, p=1, weights=distance; total time=  15.5s
[CV] END ..............n_neighbors=21, p=1, weights=distance; total time=  16.3s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=  10.0s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=  10.3s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=  10.7s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=  10.4s
[CV] END ...............n_neighbors=21, p=2, weights=uniform; total time=  10.8s
[CV] END ..............n_neighbors=21, p=2, weights=distance; total time=   9.7s
[CV] END ..............n_neighbors=21, p=2, weights=distance; total time=  10.8s
[CV] END ..............n_neighbors=21, p=2, weights=distance; total time=  10.6s
[CV] END ..............n_neighbors=21, p=2, weights=distance; total time=   9.8s
[CV] END ..............n_nei

[CV] END ..............n_neighbors=26, p=1, weights=distance; total time=  16.0s
[CV] END ...............n_neighbors=26, p=2, weights=uniform; total time=   9.5s
[CV] END ...............n_neighbors=26, p=2, weights=uniform; total time=   9.6s
[CV] END ...............n_neighbors=26, p=2, weights=uniform; total time=  10.3s
[CV] END ...............n_neighbors=26, p=2, weights=uniform; total time=  10.4s
[CV] END ...............n_neighbors=26, p=2, weights=uniform; total time=  10.7s
[CV] END ..............n_neighbors=26, p=2, weights=distance; total time=   9.9s
[CV] END ..............n_neighbors=26, p=2, weights=distance; total time=   9.9s
[CV] END ..............n_neighbors=26, p=2, weights=distance; total time=  10.0s
[CV] END ..............n_neighbors=26, p=2, weights=distance; total time=  10.1s
[CV] END ..............n_neighbors=26, p=2, weights=distance; total time=  10.5s
[CV] END ...............n_neighbors=27, p=1, weights=uniform; total time=  16.0s
[CV] END ...............n_ne

[CV] END ...............n_neighbors=31, p=2, weights=uniform; total time=   8.8s
[CV] END ...............n_neighbors=31, p=2, weights=uniform; total time=   9.1s
[CV] END ...............n_neighbors=31, p=2, weights=uniform; total time=   8.8s
[CV] END ...............n_neighbors=31, p=2, weights=uniform; total time=   9.9s
[CV] END ..............n_neighbors=31, p=2, weights=distance; total time=  10.0s
[CV] END ..............n_neighbors=31, p=2, weights=distance; total time=   9.4s
[CV] END ..............n_neighbors=31, p=2, weights=distance; total time=   9.4s
[CV] END ..............n_neighbors=31, p=2, weights=distance; total time=  10.2s
[CV] END ..............n_neighbors=31, p=2, weights=distance; total time=  10.9s
[CV] END ...............n_neighbors=32, p=1, weights=uniform; total time=  17.3s
[CV] END ...............n_neighbors=32, p=1, weights=uniform; total time=  14.9s
[CV] END ...............n_neighbors=32, p=1, weights=uniform; total time=  15.7s
[CV] END ...............n_ne

[CV] END ...............n_neighbors=36, p=2, weights=uniform; total time=  10.7s
[CV] END ...............n_neighbors=36, p=2, weights=uniform; total time=  10.9s
[CV] END ..............n_neighbors=36, p=2, weights=distance; total time=   9.8s
[CV] END ..............n_neighbors=36, p=2, weights=distance; total time=  10.1s
[CV] END ..............n_neighbors=36, p=2, weights=distance; total time=  10.1s
[CV] END ..............n_neighbors=36, p=2, weights=distance; total time=  10.0s
[CV] END ..............n_neighbors=36, p=2, weights=distance; total time=   9.7s
[CV] END ...............n_neighbors=37, p=1, weights=uniform; total time=  16.4s
[CV] END ...............n_neighbors=37, p=1, weights=uniform; total time=  16.6s
[CV] END ...............n_neighbors=37, p=1, weights=uniform; total time=  17.0s
[CV] END ...............n_neighbors=37, p=1, weights=uniform; total time=  16.4s
[CV] END ...............n_neighbors=37, p=1, weights=uniform; total time=  17.0s
[CV] END ..............n_nei

[CV] END ..............n_neighbors=41, p=2, weights=distance; total time=  10.4s
[CV] END ..............n_neighbors=41, p=2, weights=distance; total time=  10.5s
[CV] END ..............n_neighbors=41, p=2, weights=distance; total time=  10.1s
[CV] END ..............n_neighbors=41, p=2, weights=distance; total time=  10.1s
[CV] END ..............n_neighbors=41, p=2, weights=distance; total time=  10.3s
[CV] END ...............n_neighbors=42, p=1, weights=uniform; total time=  16.6s
[CV] END ...............n_neighbors=42, p=1, weights=uniform; total time=  17.2s
[CV] END ...............n_neighbors=42, p=1, weights=uniform; total time=  16.9s
[CV] END ...............n_neighbors=42, p=1, weights=uniform; total time=  17.0s
[CV] END ...............n_neighbors=42, p=1, weights=uniform; total time=  17.0s
[CV] END ..............n_neighbors=42, p=1, weights=distance; total time=  16.6s
[CV] END ..............n_neighbors=42, p=1, weights=distance; total time=  16.8s
[CV] END ..............n_nei

[CV] END ..............n_neighbors=46, p=2, weights=distance; total time=   7.9s
[CV] END ..............n_neighbors=46, p=2, weights=distance; total time=   7.8s
[CV] END ..............n_neighbors=46, p=2, weights=distance; total time=   8.0s
[CV] END ...............n_neighbors=47, p=1, weights=uniform; total time=  11.8s
[CV] END ...............n_neighbors=47, p=1, weights=uniform; total time=  11.9s
[CV] END ...............n_neighbors=47, p=1, weights=uniform; total time=  12.0s
[CV] END ...............n_neighbors=47, p=1, weights=uniform; total time=  11.9s
[CV] END ...............n_neighbors=47, p=1, weights=uniform; total time=  12.0s
[CV] END ..............n_neighbors=47, p=1, weights=distance; total time=  11.6s
[CV] END ..............n_neighbors=47, p=1, weights=distance; total time=  12.0s
[CV] END ..............n_neighbors=47, p=1, weights=distance; total time=  11.7s
[CV] END ..............n_neighbors=47, p=1, weights=distance; total time=  11.9s
[CV] END ..............n_nei

# 2. 학습

In [4]:
# knn 모델 학습
start = time()
knn_base = KNeighborsClassifier(n_neighbors=k_knn_base['n_neighbors'],
                               weights = k_knn_base['weights'],
                               p = k_knn_base['p']) # 45
knn_base.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = knn_base.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 0.005001544952392578
              precision    recall  f1-score   support

           0       0.54      0.73      0.62      7276
           1       0.55      0.36      0.43      5052
           2       0.43      0.35      0.39      4553

    accuracy                           0.52     16881
   macro avg       0.51      0.48      0.48     16881
weighted avg       0.51      0.52      0.50     16881

[[5306  832 1138]
 [2205 1822 1025]
 [2254  687 1612]]
0.5177418399383923
0.5177418399383923


# 3. 모델 저장

In [5]:
from joblib import dump, load
dump(knn_base, '../model/KNN_base_MinMax.joblib')

# # 모델 불러오기
# model = load('../model/KNN_base_MinMax.joblib')

['../model/KNN_base_MinMax.joblib']

# 파라미터 랜덤하게 늘려보기

아무래도 5만개의 데이터를 가지고 k를 1~50개만 생각해본건 너무 적은 주변 데이터를 사용한 거 같아서 다시 해봄.

In [8]:
# knn 모델 학습
start = time()
knn_random = KNeighborsClassifier(n_neighbors=100,
                               weights = 'distance',
                               p = 2) 
knn_random.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = knn_base.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 0.022010087966918945
              precision    recall  f1-score   support

           0       0.54      0.73      0.62      7276
           1       0.55      0.36      0.43      5052
           2       0.43      0.35      0.39      4553

    accuracy                           0.52     16881
   macro avg       0.51      0.48      0.48     16881
weighted avg       0.51      0.52      0.50     16881

[[5306  832 1138]
 [2205 1822 1025]
 [2254  687 1612]]
0.5177418399383923
0.5177418399383923


In [13]:
knn = KNeighborsClassifier()

myParams = {
    'n_neighbors' : [50,60,70,80,90],
    'weights' : ['uniform', 'distance'],
    'p' : [2]
}
grid = GridSearchCV(knn, myParams, cv = 5, verbose = 2)
grid.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
new_k = grid.best_params_
print(new_k)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ...............n_neighbors=50, p=2, weights=uniform; total time=  16.0s
[CV] END ...............n_neighbors=50, p=2, weights=uniform; total time=  16.9s
[CV] END ...............n_neighbors=50, p=2, weights=uniform; total time=  11.3s
[CV] END ...............n_neighbors=50, p=2, weights=uniform; total time=   9.5s
[CV] END ...............n_neighbors=50, p=2, weights=uniform; total time=   9.5s
[CV] END ..............n_neighbors=50, p=2, weights=distance; total time=   8.7s
[CV] END ..............n_neighbors=50, p=2, weights=distance; total time=   8.8s
[CV] END ..............n_neighbors=50, p=2, weights=distance; total time=   9.2s
[CV] END ..............n_neighbors=50, p=2, weights=distance; total time=   9.4s
[CV] END ..............n_neighbors=50, p=2, weights=distance; total time=   9.0s
[CV] END ...............n_neighbors=60, p=2, weights=uniform; total time=   9.0s
[CV] END ...............n_neighbors=60, p=2, wei

n개수를 훨씬 늘려야할 거 같고 p=2(유클리디언 거리), weights는 distance로 즉, 거리 가중치를 이용하는 것이 맞아 보임.

In [18]:
# 계속해서 큰 값으로 확인
# 가중치와 p는 고정함.
knn = KNeighborsClassifier(weights = 'distance', p=2)

myParams = {
    'n_neighbors' : range(100,1001,100), # 좀 큰 텀으로 찾아보기
}
grid = GridSearchCV(knn, myParams, cv = 5, verbose = 2)
grid.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
new_k = grid.best_params_
print(new_k)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ....................................n_neighbors=100; total time=   8.6s
[CV] END ....................................n_neighbors=100; total time=   9.2s
[CV] END ....................................n_neighbors=100; total time=   9.4s
[CV] END ....................................n_neighbors=100; total time=   9.6s
[CV] END ....................................n_neighbors=100; total time=   9.7s
[CV] END ....................................n_neighbors=200; total time=   9.7s
[CV] END ....................................n_neighbors=200; total time=   9.8s
[CV] END ....................................n_neighbors=200; total time=   9.5s
[CV] END ....................................n_neighbors=200; total time=   8.9s
[CV] END ....................................n_neighbors=200; total time=   9.3s
[CV] END ....................................n_neighbors=300; total time=   9.1s
[CV] END ....................................n_n

In [8]:
# knn 모델 학습
start = time()
knn_random = KNeighborsClassifier(n_neighbors=100,
                               weights = 'distance',
                               p = 2) 
knn_random.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = knn_random.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 0.0065495967864990234
              precision    recall  f1-score   support

           0       0.55      0.74      0.63      7276
           1       0.56      0.34      0.43      5052
           2       0.43      0.38      0.41      4553

    accuracy                           0.52     16881
   macro avg       0.51      0.49      0.49     16881
weighted avg       0.52      0.52      0.51     16881

[[5372  728 1176]
 [2212 1730 1110]
 [2177  631 1745]]
0.5240803269948463
0.5240803269948463


In [20]:
# 100 근처에서 찾아보자.
# 계속해서 큰 값으로 확인
# 가중치와 p는 고정함.
knn = KNeighborsClassifier(weights = 'distance', p=2)

myParams = {
    'n_neighbors' : [100], # 좀 큰 텀으로 찾아보기
}
grid = GridSearchCV(knn, myParams, cv = 5, verbose = 2)
grid.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
new_k = grid.best_params_
print(new_k)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ....................................n_neighbors=100; total time=   8.6s
[CV] END ....................................n_neighbors=100; total time=   8.8s
[CV] END ....................................n_neighbors=100; total time=   8.6s
[CV] END ....................................n_neighbors=100; total time=   9.3s
[CV] END ....................................n_neighbors=100; total time=   9.7s
베이스 모델의 파라미터 찾기 : 99.31333327293396
{'n_neighbors': 100}


In [10]:
# knn 모델 학습
start = time()
knn_random = KNeighborsClassifier(n_neighbors=100,
                               weights = 'uniform',
                               p = 2) 
knn_random.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = knn_random.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 0.00599360466003418
              precision    recall  f1-score   support

           0       0.55      0.74      0.63      7276
           1       0.56      0.33      0.42      5052
           2       0.43      0.38      0.40      4553

    accuracy                           0.52     16881
   macro avg       0.51      0.48      0.48     16881
weighted avg       0.52      0.52      0.51     16881

[[5411  699 1166]
 [2251 1672 1129]
 [2197  631 1725]]
0.521770037320064
0.521770037320064


결과가 계속 비슷한 것으로 보아 n의 개수가 그다지 큰 성능 차이를 보이진 않는 듯 하다.

In [11]:
# knn 모델 학습
start = time()
knn_random = KNeighborsClassifier(n_neighbors=1000) 
knn_random.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = knn_random.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 0.004006385803222656
              precision    recall  f1-score   support

           0       0.54      0.75      0.63      7276
           1       0.61      0.25      0.35      5052
           2       0.41      0.44      0.43      4553

    accuracy                           0.51     16881
   macro avg       0.52      0.48      0.47     16881
weighted avg       0.53      0.51      0.49     16881

[[5428  475 1373]
 [2352 1242 1458]
 [2222  335 1996]]
0.5133582133759849
0.5133582133759849


In [15]:
# 

from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()

myParams = {
    'n_neighbors' : range(95,126,10),
    'weights' : ['distance', 'uniform'],
     'p' : [1,2]
}
grid = GridSearchCV(knn, myParams, cv = 3, verbose = 2)
grid.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
new_k = grid.best_params_
print(new_k)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END ..............n_neighbors=95, p=1, weights=distance; total time=  16.7s
[CV] END ..............n_neighbors=95, p=1, weights=distance; total time=  16.6s
[CV] END ..............n_neighbors=95, p=1, weights=distance; total time=  16.9s
[CV] END ...............n_neighbors=95, p=1, weights=uniform; total time=  16.9s
[CV] END ...............n_neighbors=95, p=1, weights=uniform; total time=  16.5s
[CV] END ...............n_neighbors=95, p=1, weights=uniform; total time=  17.4s
[CV] END ..............n_neighbors=95, p=2, weights=distance; total time=  12.1s
[CV] END ..............n_neighbors=95, p=2, weights=distance; total time=  11.0s
[CV] END ..............n_neighbors=95, p=2, weights=distance; total time=  11.2s
[CV] END ...............n_neighbors=95, p=2, weights=uniform; total time=  11.3s
[CV] END ...............n_neighbors=95, p=2, weights=uniform; total time=  11.2s
[CV] END ...............n_neighbors=95, p=2, wei

In [20]:
# 교차검증을 3번으로 줄여서 확실하진 않지만 p=1인 경우도 최적의 경우라고 판단할 때가 있음
# knn 모델 학습
start = time()
knn_random = KNeighborsClassifier(n_neighbors=115,
                               weights = 'distance',
                               p = 1,
                               algorithm = 'auto') 
knn_random.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = knn_random.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 0.006997823715209961
              precision    recall  f1-score   support

           0       0.55      0.74      0.63      7276
           1       0.58      0.35      0.43      5052
           2       0.44      0.39      0.41      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.49      0.49     16881
weighted avg       0.53      0.53      0.51     16881

[[5389  696 1191]
 [2225 1750 1077]
 [2214  555 1784]]
0.5285824299508323
0.5285824299508323


In [24]:
# 

from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()

myParams = {
    'n_neighbors' : range(110,120,2),
    'weights' : ['distance'],
     'p' : [1,2]
}
grid = GridSearchCV(knn, myParams, cv = 3, verbose = 2)
grid.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델의 파라미터 찾기 : {time()-start}')
new_k = grid.best_params_
print(new_k)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .............n_neighbors=110, p=1, weights=distance; total time=  17.3s
[CV] END .............n_neighbors=110, p=1, weights=distance; total time=  17.4s
[CV] END .............n_neighbors=110, p=1, weights=distance; total time=  17.0s
[CV] END .............n_neighbors=110, p=2, weights=distance; total time=  11.4s
[CV] END .............n_neighbors=110, p=2, weights=distance; total time=  11.3s
[CV] END .............n_neighbors=110, p=2, weights=distance; total time=  11.7s
[CV] END .............n_neighbors=112, p=1, weights=distance; total time=  16.9s
[CV] END .............n_neighbors=112, p=1, weights=distance; total time=  16.9s
[CV] END .............n_neighbors=112, p=1, weights=distance; total time=  18.0s
[CV] END .............n_neighbors=112, p=2, weights=distance; total time=  12.2s
[CV] END .............n_neighbors=112, p=2, weights=distance; total time=  12.8s
[CV] END .............n_neighbors=112, p=2, weig

In [25]:
# 교차검증을 3번으로 줄여서 확실하진 않지만 p=1인 경우도 최적의 경우라고 판단할 때가 있음
# knn 모델 학습
start = time()
knn_random = KNeighborsClassifier(n_neighbors=new_k['n_neighbors'],
                               weights = 'distance',
                               p = new_k['p'],
                               algorithm = 'auto') 
knn_random.fit(X_train_base_scaled, y_train_base)
print(f'베이스 모델 : {time()-start}')

# 검증 데이터를 이용한 예측
y_val_pred = knn_random.predict(X_val_base_scaled)

# 성능 평가
report = classification_report(y_val, y_val_pred)
print(report)
print(confusion_matrix(y_val, y_val_pred))
print(f1_score(y_val, y_val_pred,average='micro'))
print(accuracy_score(y_val, y_val_pred))

베이스 모델 : 0.0035338401794433594
              precision    recall  f1-score   support

           0       0.55      0.74      0.63      7276
           1       0.58      0.35      0.43      5052
           2       0.44      0.39      0.41      4553

    accuracy                           0.53     16881
   macro avg       0.52      0.49      0.49     16881
weighted avg       0.53      0.53      0.51     16881

[[5386  692 1198]
 [2228 1744 1080]
 [2216  559 1778]]
0.527693856998993
0.527693856998993
